In [ ]:
import os

# Read the files specified above
contents = []
for fname in os.listdir("data/lintulehtio"):
    if not "nayte" in fname:
        continue
    path = os.path.join("data/lintulehtio", fname)
    with open(path, 'r') as f:
        contents.append((fname, "".join(f.readlines())))

# temporarily decrease amount of texts
# contents = contents[:2]

for fname, text in contents:
    print(f"{fname}:\n")
    print(f"{text}\n")

In [ ]:
# koodit
codes = ['Tunteet', 'Suojeleminen', 'Kiireettömyys', 'Ristiriita', 'Puutarha', 'Kesä', 'Omakotitalo', 'Havainnointi', 'Muisti', 'Lapsuus', 'Nostalgia', 'Rakennukset', 'Moottoritie', 'Rauha', 'Ympäristö', 'Hiljaisuus', 'Linnut', 'Muutos', 'Luonto']

In [ ]:
import json
import requests
from pprint import pprint

from llm import generate

output_format = {
    "type": "object",
    "properties": {
        "code_present": {
            "type": "boolean"
        }
    },
    "required": [
      "code_present"
    ]
}
results = []
n_iter = 10
for fname, text in contents:
    for code in codes:
        for idx in range(n_iter):
            messages = [
                {
                    "role": "system",
                    "content": "Olet laadullisen tutkimuksen avustaja. Saat tekstinäytteen sekä aineiston pohjalta rakennetun koodikirjan yksittäisen koodin. Lue teksti huolella ja päätä kuvaako koodi tekstinäytettä."    
                },
                {
                    "role": "user",
                    "content": f"Koodi: {code} \n\n Tekstinäyte: \n\n {text}"
                }
            ]
            # model = "mistral-large"
            # model = "deepseek-r1:70b"
            model = "llama3.3:70b"
            result = generate(model, messages, seed=idx, n_context=4096, output_format=output_format)
        
            code_present = json.loads(result['message']['content'])['code_present']
            results.append({
                "fname": fname,
                "code": code,
                "iter": idx,
                "result": code_present
            })

In [ ]:
%pip install -q jinja2
import pandas as pd

# Create a dictionary to store the results
transformed_data = {}

# Process the results
for item in results:
    fname = item['fname']
    code = item['code']
    
    if fname not in transformed_data:
        transformed_data[fname] = {}
    
    if code not in transformed_data[fname]:
        transformed_data[fname][code] = []
    
    transformed_data[fname][code].append(item['result'])

# Calculate percentages
for fname in transformed_data:
    for code in transformed_data[fname]:
        true_count = transformed_data[fname][code].count(True)
        total_count = len(transformed_data[fname][code])
        transformed_data[fname][code] = (true_count / total_count) * 100

# Create DataFrame
df = pd.DataFrame.from_dict(transformed_data, orient='index')

# Add averages
df['total'] = df.mean(axis=1)
df.loc['total'] = df.mean()

# Format percentages
df = df.round(2)

# Define the styling function
def color_high_values(val):
    color = 'background-color: rgba(144, 238, 144, 0.3)' if val >= 80 else ''
    return color

# Apply the styling
styled_df = df.style.map(color_high_values).format("{:.2f}")

# Display the styled DataFrame
styled_df